<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/Semana%201/LDA_ejercicios_Clasificaci%C3%B3n_de_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencias

In [ ]:
%%capture
!pip install -U plotly

In [ ]:
from pprint import pprint

import pandas as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix

import plotly.express as px

In [ ]:
sep= "\n-------------------\n"

# Datos

Trabajaremos con datos del Lending Club.

Lending Club es una plataforma de préstamos entre pares (P2P), donde los prestatarios envían sus solicitudes de préstamo y los prestamistas individuales seleccionan las solicitudes que desean financiar. Los prestatarios reciben el monto total del préstamo emitido menos la tarifa inicial, que se paga a la empresa. Los inversores compran notas respaldadas por préstamos personales y pagan a Lending Club una tarifa de servicio.

Los préstamos P2P reducen el costo de los préstamos personales en comparación con el financiamiento tradicional al conectar directamente a los prestatarios e inversores. Sin embargo, siempre existe el riesgo de invertir en un préstamo incobrable. De hecho, la tasa de incumplimiento de los préstamos P2P es mucho más alta que la de los préstamos tradicionales. Por lo tanto, la industria crediticia está muy interesada en brindar a los inversionistas una evaluación integral del riesgo de las solicitudes de préstamo. La empresa comparte datos sobre todas las solicitudes de préstamos realizadas a través de su plataforma.

La descripción de las variables en el dataset se puede descargar [aqui](http://www-2.rotman.utoronto.ca/~hull/mlbook/lendingclub_datadictionary.xlsx).


In [ ]:
!mkdir data
!wget http://www-2.rotman.utoronto.ca/~hull/mlbook/lending_clubFull_Data_Set.xlsx -O data/lending_club.xlsx

In [ ]:
data_raw = pd.read_excel("data/lending_club.xlsx", index_col=0)
data_raw

## Datos Faltantes

In [ ]:
# DataFrame o Series en la que aparezca el número de datos faltantes

In [ ]:
px.bar(na_values, "index", "n")

In [ ]:
data_filt = data_raw.loc[:, data_raw.columns[na_values.n < .1]]
data_filt

In [ ]:
print("Datos originales\n")
print(data_raw.dtypes.value_counts())
print(sep)
print("Datos filtrado\n")
data_filt.dtypes.value_counts()

## División del dataset en entrenamiento y pruebas

In [ ]:
data_filt.loan_status.value_counts(dropna = False)

In [ ]:
loan_status_to_objective = {
    "Current": 0, 
    "Fully Paid": 0, 
    "Charged Off": 1, 
    "Late (31-120 days)": 0, 
    "In Grace Period": 0, 
    "Late (16-30 days)": 0,  
    "Default": 1, 
}

X = data_filt.drop('loan_status', 1)
y = data_filt.loan_status.map(loan_status_to_objective)

X = X[~y.isna()]
y = y[~y.isna()]
print(X.shape, y.shape)
y.dtype

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .7, random_state = 10)

# Análisis de las variables por tipo

In [ ]:
columns_by_type = {k.name: v for k, v in X.columns.to_series().groupby(data_filt.dtypes).groups.items()}
pprint(columns_by_type)

In [ ]:
X.select_dtypes('float64')

## Variables Numéricas


### Variables con poca variación

In [ ]:
var_filter = VarianceThreshold(threshold=.90)
var_filter.fit(X_train[columns_by_type['float64']])
constant_columns = [column for column in X_train[columns_by_type['float64']].columns
                    if column not in X_train[columns_by_type['float64']].columns[var_filter.get_support()]]
constant_columns

In [ ]:
X_train.drop(columns=constant_columns, inplace=True)
X_test.drop(columns=constant_columns, inplace=True)

columns_by_type['float64'] = columns_by_type['float64'].drop(constant_columns)

In [ ]:
scaler = StandardScaler()
data_num_sc = pd.DataFrame(scaler.fit_transform(X_train[columns_by_type['float64']]), columns = X_train[columns_by_type['float64']].columns, index = X_train[columns_by_type['float64']].index)
data_num_sc

### Variables con alta correlación

In [ ]:
numeric_to_remove = []

In [ ]:
# Graficar la matriz de correlación de las variables de tipo float64


In [ ]:
correlated_features = set()
for i in range(len(corr_df.columns)):
    for j in range(i):
        if abs(corr_df.iloc[i, j]) > 0.7:
            colname = corr_df.columns[i]
            correlated_features.add(colname)
len(correlated_features)

In [ ]:
correlated_features

In [ ]:
X_train.drop(columns=correlated_features, inplace=True)
X_test.drop(columns=correlated_features, inplace=True)

data_num_sc.drop(columns=correlated_features, inplace=True)
columns_by_type['float64'] = columns_by_type['float64'].drop(correlated_features)


## Fechas

Las fechas no las ocuparemos a menos en este análisis. Aunque cabe destacar que podrían ocupar si se transforman a otro tipo de dato (e.g. crear "número de días desde..." y obtener la nueva variable usando una diferencia en días entre fechas).

In [ ]:
for col in columns_by_type["datetime64[ns]"]:
    print(data_filt[col].head(), sep)

In [ ]:
X_train.drop(columns=columns_by_type["datetime64[ns]"], inplace=True)
X_test.drop(columns=columns_by_type["datetime64[ns]"], inplace=True)

## Factores

Para poder usar factores en el modelo, es mecesario convertirlas a variables dummies. Esto es, considerando la siguiente variable:

estado_civil|
------------|
soltero
casado
soltero
soltero
viudo

Al obtener las variables dummies de esta obtendriamos:

estado_civil_soltero|estado_civil_casado|estado_civil_viudo
---|---|---
1|0|0
0|1|0
1|0|0
1|0|0
0|0|1

Incluso se puede quitar uno de los niveles y dejarlo como el estado base:

estado_civil_casado|estado_civil_viudo
---|---
0|0
1|0
0|0
0|0
0|1


Si nuestra variable tiene muchos niveles, el crear variables dummies de esta puede hacer que nuestro dataset crezca en dimensión, complicando el entrenamiento del modelo. Para estos casos, se puede buscar la posibilidad de unir distintos niveles en uno sólo o eliminar la variable.

In [ ]:
unique_values_by_column = X_train[columns_by_type["object"]].nunique().reset_index(name = "n")
unique_values_by_column

In [ ]:
px.bar(data_frame = unique_values_by_column, 
       x ="index", 
       y = "n", 
       title="Cantidad de niveles por factor", 
       labels={
           "index": "Nombre de la variable",
           "n": "Número de niveles"
           }
       )

Eliminamos las variables con más de 900 niveles.

In [ ]:
drop_columns = columns_by_type["object"][unique_values_by_column.n > 900].to_list()
drop_columns

In [ ]:
X_train.drop(columns=drop_columns, inplace=True)
X_test.drop(columns=drop_columns, inplace=True)

columns_by_type['object'] = columns_by_type['object'].drop(drop_columns)

In [ ]:
X_train

In [ ]:
 X_train = pd.get_dummies(X_train, columns=columns_by_type['object'])
 X_train

# Imputación de datos faltantes

Como los modelos matemáticos no trabajan con valores faltantes, es necesario aplicar un tratamiento a estos, ya sea eliminando los casos o imputándolos. Para imputarlos podemos optar por métodos sencillos (media, mediana o moda de la variable) o ir por métodos un poco más elaborados (Multiple Imputation by Chained Equations, KNN, Exact Matrix Completion via Convex Optimization, etc.). 

Lo recomendable es hacer un análisis de los datos faltantes por variable antes de pensar en imputar los datos (ver: [To impute or not to impute?](https://towardsdatascience.com/to-impute-or-not-to-impute-a-practical-example-when-imputation-could-lead-to-wrong-conclusions-fd1e340d779a)).

En este caso, imputamos los valores faltantes en nuestro dataset de la manera más ingenua posible. Ponemos 99 en todos los valores faltantes. En los ejercicios vamos a pensar un poco sobre las implicaciones de esta imputación.

In [ ]:
X_train_fact = X_train.fillna(99).drop("id", 1)
X_train_fact

In [ ]:
px.bar(pd.concat([X_train_fact, y_train], 1).corr()['loan_status'])

In [ ]:
data_plot = pd.concat([X_train_fact, y_train], 1)
data_plot.loan_status = data_plot.loan_status + np.random.normal(0, .1, len(data_plot))

In [ ]:
px.scatter(data_frame = data_plot, x = 'loan_status', y = 'last_fico_range_high', opacity = .5, title = 'last_fico_range_high vs. loan_status <br><span>Tiene ruido para facilitar la visualización</span>')

# Latent Discriminant Analysis

## Entrenamiento

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_fact, y_train)
train_pred = lda.predict(X_train_fact)
print(classification_report(y_train, train_pred))

Imaginemos que nuestro vector de etiquetas se ve así (0, 1, 1, 0, 1), estas son las etiquetas **reales**, y que el vector de **predicciones** se ve así (0, 0, 1, 0, 0).

real: (0, 1, 1, 0, 1) \
pred: (0, 0, 1, 0, 0)


La matriz de confusión para este caso se vería de la siguiente manera

*|1|0|
-|-|-|
**1**|1|2|
**0**|0|2|


*| no me pago (1) | me pago (0)
-----|-|-|
no me pago (1) | 1332 | 470
me pago (0) | 526 | 15148

In [ ]:
confusion_matrix(y_train, train_pred)

## Prueba

In [ ]:
X_test

In [ ]:
X_test_fact = pd.get_dummies(X_test, columns=columns_by_type['object']).fillna(99)
X_test_fact

In [ ]:
missing_cols = set( X_train_fact.columns ) - set( X_test_fact.columns )
missing_cols

In [ ]:
for c in missing_cols:
    X_test_fact[c] = 0
X_test_fact = X_test_fact[X_train_fact.columns]
X_test_fact

In [ ]:
test_pred = lda.predict(X_test_fact)
print(classification_report(y_test, test_pred))

In [ ]:
print(classification_report(y_train, train_pred))

Comparación de precisión obtenida en el set de entrenamiento y el set de prueba.

In [ ]:
print(
    lda.score(X_train_fact, y_train), 
    lda.score(X_test_fact, y_test)
)

## ¿Cómo afecta cada variable?

In [ ]:
coef_df = pd.DataFrame(lda.coef_[0], X_train_fact.columns, ["coef_"])
print(lda.intercept_)
coef_df

In [ ]:
px.bar(coef_df.reset_index(), 'coef_', 'index')

In [ ]:
X_test.loan_amnt * 10e-6

In [ ]:
y_train[train_pred == 0]

In [ ]:
np.dot(lda.coef_, X_train_fact.loc[9377, :]) + lda.intercept_

In [ ]:
y_train[train_pred == 1]

In [ ]:
np.dot(lda.coef_, X_train_fact.loc[14719, :]) + lda.intercept_

## LDA para reducir dimensión

In [ ]:
X_proj = lda.transform(X_train_fact)
X_proj.shape

In [ ]:
X_proj

In [ ]:
probs = lda.predict_proba(X_train_fact)

In [ ]:
probs[0]

In [ ]:
tol = 1e-3
for i, p in enumerate(probs):
    if .5-tol < p[0] < .5 + tol:
        print(i, p)

In [ ]:
line = X_proj[4558]
line[0]

In [ ]:
fig = px.scatter()
fig.add_scatter(x = X_proj[y_train == 0, 0], y = np.random.rand(len(X_proj[y_train==0])), mode = "markers", opacity=.5, name = "normal")
fig.add_scatter(x = X_proj[y_train == 1, 0], y = np.random.rand(len(X_proj[y_train==1])), mode = "markers", opacity = .5, name = "default")
fig.add_vline(x = line[0], line_dash = 'dash')

fig.update_layout(
    title="Reducción de la dimensión de LDA",
    xaxis_title="LDA1<br><sup>Pese a que los datos se muestran en dos dimensiones, la proyección sólo fue a una (= n_clases - 1)<br>Se agrego aleatoriedad al eje y para facilitar la visualización</sup>",
    legend_title="Etiqueta",
    font=dict(
        size=18
    )
)

fig.show()

# Ejercicios


## Factores

### Condensar niveles

Condensar niveles de factores usualmente es antecedido por un análisis por variable comparando la relación entre cada nivel, la variable de respuesta (en este caso ```loan_status```) y otras covariables. De este modo, niveles que aparentan tener la misma relación con la variable de respuesta pueden juntarse en uno solo; Algunas veces, mejorando el desempeño del modelo y reduciendo la dimensionalidad (y con esto el tiempo de cómputo en algunos casos).

Sin embargo, si se tiene una gran cantidad de factores como covariables y estos a su vez tienen muchos niveles, puede llegar a ser un trabajo arduo que no compense la mejora obtenida en el desempeño del modelo o la rapidez del entrenamiento.

Para este ejercicio, use su intuición en lugar de hacer estas comparaciones. Tampoco es necesario considerar todos los factores, con una cantidad pequeña (por ejemplo, a lo más 3) es suficiente.

1. Condense los niveles de factores que usted considere pueden pertenecer a uno sólo. Puede comenzar con la variable ```home_ownership```, condensando en un nivel llamado ```not_owned``` a todos los niveles que no son ```OWN```.
2. Entrene LDA con estos cambios y compare los resultados obtenidos. ¿Mejoró el modelo?

## Imputación (opcional)

### Constante

1. ¿Qué problemas tiene imputar un valor constante en todas las variables con datos faltantes? Piense en la escala de las variables
    2. ¿Cómo afecta a la estandarización de las variables?
    3. ¿Qué relación tiene con los outliers?
2. ¿Por qué en este caso no presentó problemas aparentes?

### Media, mediana y moda

1. Cambiar la imputación realizada por la media, mediana y moda (tres ejercicios diferentes, realizando uno esta bien) y comparar los resultados obtenido con los vistos en el notebook.
2. ¿Qué implica llenar los valores valores vacios con este tipo de valores? Tome como ejemplo la variable ```mths_since_last_delinq```, ¿qué asumimos de las personas que no han delinquido?

Nota: Para familiarizarse con pandas, es un buen ejercicio realizarlo utilizando sólo pandas. Pero también puede utilizar [```sklearn.impute.SimpleImputer```](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

### KNN (más opcional)

1. Realice la imputación de los valores con KNN y compare los resultados obtenidos.
2. ¿Cómo funciona esta imputación y qué implica para los valores perdidos?

Nota: Puede usar la implementación realizada en [```fancyimpute```](https://github.com/iskandr/fancyimpute)

# Ligas interesantes

- [BASIC LITERACY OF STATISTICS — 3](https://medium.com/@yohoshiva1609/basic-literacy-of-statistics-3-bc9f5a69f116)
- [Sobre la estandarización en variables dummies](https://stats.stackexchange.com/questions/463690/multiple-regression-with-mixed-continuous-categorical-variables-dummy-coding-s)

## TODO

- Agregar liga a matriz de confusion y metricas